In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd

In [3]:
from beads.simple import SimpleExecutor
from beads.composed import ComposedExecutor
from beads.features import FeatureExtractor

In [4]:
from beads.parser import Parser

In [11]:
from beads import execute

In [10]:
query_events_df = pd.read_csv('events.csv')

In [36]:
query_events_df.head()

,user_id,time,type
0,0313cfb4506962bff822094b262781615818ed502418f6...,1705955544,first_open
1,0313cfb4506962bff822094b262781615818ed502418f6...,1705955545,session_start
2,0313cfb4506962bff822094b262781615818ed502418f6...,1705955545,screen_view
3,0313cfb4506962bff822094b262781615818ed502418f6...,1705955546,firebase_campaign
4,0313cfb4506962bff822094b262781615818ed502418f6...,1705955586,IntroStarted


# Queries

Sessions with ConnectionError during the intro cutscene
```
session_start 
=> IntroStarted
=> not {IntroCompleted} 
=> ConnectionError
```

Sessions with multiple - 3x - ConnectionError after intro cutscene
```
session_start
=> IntroStarted
=> not {ConnectionError}
=> IntroCompleted
=> error: ConnectionError
=> ConnectionError[this.event_occurence - error.event_occurence == 2]
```

In [25]:
query_str = """
start: session_start
=> IntroStarted
=> not {ConnectionError}
=> IntroCompleted
=> test: ConnectionError
=> ConnectionError[this.event_occurence - test.event_occurence == 2]
"""

In [26]:
response = execute(query_events_df, query_str)

Tree('start', [Tree('node', [Tree('simple_node', [Tree('alias', [Token('NAME', 'start')]), Tree('event_name', [Token('NAME', 'session_start')]), Tree('conditions', [])])]), Tree('node', [Tree('simple_node', [Tree('event_name', [Token('NAME', 'IntroStarted')]), Tree('conditions', [])])]), Tree('node', [Tree('not_node', [Tree('start', [Tree('node', [Tree('simple_node', [Tree('event_name', [Token('NAME', 'ConnectionError')]), Tree('conditions', [])])])])])]), Tree('node', [Tree('simple_node', [Tree('event_name', [Token('NAME', 'IntroCompleted')]), Tree('conditions', [])])]), Tree('node', [Tree('simple_node', [Tree('alias', [Token('NAME', 'test')]), Tree('event_name', [Token('NAME', 'ConnectionError')]), Tree('conditions', [])])]), Tree('node', [Tree('simple_node', [Tree('event_name', [Token('NAME', 'ConnectionError')]), Tree('conditions', [Token('CONDITION', 'this.event_occurence - test.event_occurence == 2')])])])])
--------------
Tree('start', [Tree('node', [Tree('simple_node', [Tree('e

In [30]:
result_df = response['result']

In [34]:
result_df.loc[
    result_df['user_id'].drop_duplicates().index
].head()

,user_id,event_e_0,event_e_1,event_e_3,event_e_4,event_e_5,event_e_2_not_0
0,0019b19acc451c16754116077a725316714baf9307e16e...,2,7,8,9,11,NaN
657,0044a0f4c0b1a38bdbcabc11eba7a4f31ee31a3c8ac9a6...,1,8,11,25,27,NaN
659,0061026a354e30f6bd1e2f83abf7de689f5ac3a265a43c...,1,11,13,14,16,NaN
796,00939f3ffaa598859dedea6dae8edf842ff2447237b6ed...,1,39,42,48,50,NaN
873,00d436741348173fef1fc0f112307b2353cb3b349cbf7b...,2,20,21,22,24,NaN


# Checks

In [28]:
df = response['events']

In [38]:
df[
    df['user_id'].str.contains('0019b19acc451c16754116077a725316714baf9307e16e')
].sort_values('time').iloc[0:20][['event_order_id', 'type', 'time']]

,event_order_id,type,time
event_index,,,
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_0,0,first_open,1706188328
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_1,1,screen_view,1706188330
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_2,2,session_start,1706188330
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_3,3,firebase_campaign,1706188342
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_4,4,ResolveUrlSuccess,1706188363
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_5,5,ResolveUrlSuccess,1706188366
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_6,6,ResolveUrlTimeLoading,1706188366
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_7,7,IntroStarted,1706188373
0019b19acc451c16754116077a725316714baf9307e16e7c36fc0376c477bbd6_8,8,IntroCompleted,1706188389
